In [25]:
import numpy as np
from random import random

#save the activations and derivatives
#implement backpropagation
#implemenet gradient descent
#implement train 
#train our net with some dummy dataset
#make some predictions

class MLP(object):
    """A Multilayer Perceptron class
    """
    
    #constructor
    def __init__(self,num_inputs = 3,num_hidden = [3,3],num_outputs=2):
        """Constructor for the MLP. Takes the number of inputs,
        a variable number of hidden layers, and number of outputs
        
        Args:
            num inputs (int): Number of inputs
            hidden_layers (list): A list of ints for the hidden layers
            num_outputs (int): Number of outputs
        
        """
        
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        
        #create a generic representation of the layers
        layers = [self.num_inputs] + self.num_hidden + [self.num_outputs]
        
        #create random connection representation of the layers
        weights = []
        for i in range(len(layers)-1):
            w = np.random.rand(layers[i], layers[i+1])
            weights.append(w)
        self.weights = weights
        
        activations = []
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
            #instance variable
        self.activations = activations

        derivatives = []
        #derivative depends on the number of weight matrices
        for i in range(len(layers)-1):
            d = np.zeros((layers[i],layers[i+1]))
            derivatives.append(d)
            #instance variable
        self.derivatives = derivatives
        
            
    def forward_propagate(self, inputs):
        """Computes forward propagation of the network based on input signals
        Args:
            inputs (ndarry): Input signals
        Return:
            activations (ndarray): Output values        
        """
        
        #the input layer activation is just the input itself
        activations = inputs
        self.activations[0] = activations
        
        #iterate through the network layers
        for i, w in enumerate(self.weights):
            #calculate matrix multiplication between previous activation and weight
            net_inputs = np.dot(activations, w)
            
            #apply sigmoid activation function
            activations = self._sigmoid(net_inputs)
            
            #save the activation for backpropagation
            self.activations[i+1] = activations
            
        #return output layer activation
        return activations
    
    def back_propagate(self, error, verbose = False):
        
        #dE/dW_i = (y-a_[i+1]) s'(h_[i+1]) a_i
        #s'(h_[i+1]) = s(h_[i+1])(1-s(h_[i+1]))
        #s(h_[i+1]) =a_[i+1]
        
        #dE/dW_[i-1] = (y-a_[i+1]) s'(h_[i+1]) W_i s'(h_i) a_[i-1]
        
        for i in reversed(range(len(self.derivatives))):
            
            #get activation for previous layer
            activations = self.activations[i+1]
            
            #apply sigmoid derivative function
            delta = error * self._sigmoid_derivative(activations)# ndarray([0.1, 0.2])-->ndarray([[0.1],[0.2]]) 
            
            #reshape delta as to have it as a 2d array
            delta_reshaped = delta.reshape(delta.shape[0],-1).T
            
            #get activations for current layer
            current_activations = self.activations[i] # ndarray([0.1, 0.2])-->ndarray([0.1],[0.2]) 
            
            #reshape activations as to have them as a 2d column matrix
            current_activations_reshaped = current_activations.reshape(current_activations.shape[0],-1)
            
            #save derivative after applying matrix multiplication
            self.derivatives[i] = np.dot(current_activations_reshaped, delta_reshaped)
            
            #backpropagate the next error
            error = np.dot(delta,self.weights[i].T)

            if verbose:
                print("Detivatives for W{}: {}".format(i,self.derivatives[i]))
        
        
        #error propagated all the way back to the input layer
        return error
    

        
    def train(self,inputs, targets, epochs, learning_rate):
        """Trains model running forward prop and backprop
        Args:
            inputs(ndarray): X
            targets(ndarray): Y
            epochs(int): Number of epochs we want to train the network for
            learning rate(float): Step to apply to gradient descent
        """
        
        #iterate through all the training data
        for i in range(epochs):
            """epoch tells us how many times we need to feed the data to the neural network
            the more we do this, the network will make better predictions
            """
            sum_errors = 0
            
            #iterate through all the training data
            for j, inputt in enumerate(inputs):                
                target = targets[j]
                
                #forward propagation
                output = self.forward_propagate(inputt)
                
                #calculate error
                error = target - output
                
                #back propagation
                self.back_propagate(error)
                
                #apply gradient descent
                self.gradient_descent(learning_rate)
                
                #keep track of the MSE for reporting later
                sum_errors += self._mse(target, output)
            
            #Epoch complete. report the training error
            print("Error: {} at epoch {}".format(sum_errors / len(inputt), i+1))
        
        print("Training complete!")
        print("=====")

    def gradient_descent(self, learning_rate):
        for i in range(len(self.weights)):
            weights = self.weights[i]
            #print("Original W{} {}".format(i,weights))
            derivatives = self.derivatives[i]
            weights += derivatives * learning_rate
            #print("Updated W{} {}".format(i,weights))            
            
    def _sigmoid(self, x):
        """Sigmoid activation function
        Args:
            x(float): Value to be processed
        Returns:
            y(float): Output
        """
        y = 1.0/(1+np.exp(-x))
        return y
        
    def _sigmoid_derivative(self,x):
        """Sigmoid activation function
        Args:
            x(float): Value to be processed
        Returns:
            y(float): Output
        """
        return x * (1.0-x)       
    
    def _mse(self, target, output):
        return np.average((target - output)**2)    
    
if __name__ == "__main__":
    
    #create a dataset to train a network for the sum operation
    #list comprehension
    #array([[0.1, 0.2], [0.3,0.4]])
    inputs = np.array([[random() / 2 for _ in range(2)] for _ in range(1000)])
    #array([[0.3],[0.7])
    targets = np.array([[i[0] + i[1]] for i in inputs])
    
    #create an MLP
    mlp= MLP(2, [5], 1)
    
    #train our mlp
    mlp.train(inputs, targets, 50, 0.3)
    
    inputt = np.array([0.3,0.1])
    target = np.array([0.4])
    
    output=mlp.forward_propagate(inputt)
    print()
    print("Our network believes that {} + {} is equal to {}".format(inputt[0],inputt[1],output[0]))
    



Error: 21.57299786360237 at epoch 1
Error: 20.285169679266602 at epoch 2
Error: 19.81440965671707 at epoch 3
Error: 18.891760460189946 at epoch 4
Error: 17.002863803269346 at epoch 5
Error: 13.841199248802257 at epoch 6
Error: 10.077931118579306 at epoch 7
Error: 6.873145783634494 at epoch 8
Error: 4.651252710556308 at epoch 9
Error: 3.2223231372293957 at epoch 10
Error: 2.307924024620498 at epoch 11
Error: 1.7107799114352789 at epoch 12
Error: 1.3107440020011343 at epoch 13
Error: 1.0363259577235695 at epoch 14
Error: 0.8442321189951045 at epoch 15
Error: 0.7074558484777301 at epoch 16
Error: 0.6086276646343671 at epoch 17
Error: 0.5362678228438689 at epoch 18
Error: 0.4826147311922651 at epoch 19
Error: 0.44232339018706834 at epoch 20
Error: 0.4116588054957553 at epoch 21
Error: 0.3879807703951344 at epoch 22
Error: 0.36940600629523246 at epoch 23
Error: 0.3545816243953545 at epoch 24
Error: 0.3425303795845481 at epoch 25
Error: 0.3325433256878272 at epoch 26
Error: 0.324104405666617